In [3]:
#| code-fold: true

################################################################################

# autoreload all modules every time before executing the Python code
%reload_ext autoreload
%autoreload 2

################################################################################

from IPython.core.interactiveshell import InteractiveShell

# `ast_node_interactivity` is a setting that determines how the return value of the last line in a cell is displayed
# with `last_expr_or_assign`, the return value of the last expression is displayed unless it is assigned to a variable
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

################################################################################

import pandas as pd

# `copy_on_write` is a performance improvement
# This will be the default in a future version of pandas
# Refer to https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
pd.options.mode.copy_on_write = True
pd.options.future.no_silent_downcasting = True

################################################################################

%matplotlib inline

import matplotlib as mpl

mpl.use("agg")

# `constrained_layout` helps avoid overlapping elements
# Refer to https://matplotlib.org/stable/tutorials/intermediate/constrainedlayout_guide.html
mpl.pyplot.rcParams["figure.constrained_layout.use"] = True

::: {.callout-note}

The UI elements in this post are not interactive as this is a static page. To see the interactive elements, please run the notebook in a Jupyter environment.

```

In [4]:
import holoviews as hv
import hvplot.pandas  # noqa
import pandas as pd
import panel as pn  # noqa
import param as pm

pn.extension("tabulator")
hv.extension("bokeh")

[`panel`](https://panel.holoviz.org/) is a library that allows creating interactive dashboards in pure Python. It's a flexible library that allows interlinking matplotlib, bokeh, widgets, and more. 

There are however many ways to use `panel`, some of which are better for certain use cases. I wanted to write this post to share the way I use `panel`.

## Development Constraints

My first constraint when using `panel` was that I wanted to develop the dashboard in an interactive manner, preferably using a Jupyter notebook environment. `panel` does allow starting a server using the `panel serve` command and you can pass in the path to a file or a jupyter notebook. However, I only want to prototype individual components in a Jupyter notebook, and once I'm happy with the components, I want to combine them into a dashboard. For multi-page dashboards, I didn't want to have to store different components in different files and run multiple servers. 

My second constraint was that I wanted to make the dashboard as modular as possible. I wanted to be able to reuse some components across different dashboards. For example, I might have a component that shows a line chart and I want to use that line chart in a "user guide page" as well as in the "main dashboard".

Lastly, I wanted the code to be usable in a Python script in case someone wanted to programmatically access the same features. Imagine if a user wanted to plot the line chart from the dashboard and annotate it with some custom text. I wanted to make it easy for an advanced user that wanted to do that to have the option to do so.

The most natural way to do this was to make the components as classes and then instantiate them as part of the dashboard.
For this post, I'm going to use the IMDb movies dataset as an example to build a dashboard.

## Jupyter Notebook

When working in a Jupyter notebook, autoreloading modules is a feature that is very useful. This means you can make a python package and import the package in the notebook, prototype components in the notebook and move the code over to the package when you are happy with the component. With autoreloading, you don't have to restart the jupyter notebook kernel every time you make a change to the source code.

In [5]:
# autoreload all modules every time before executing the Python code
%reload_ext autoreload
%autoreload 2

For this notebook, I have a package called `movies_dashboard` that has the following structure:

In [6]:
!eza --tree src

src
└── movies_dashboard
    ├── __init__.py
    ├── movies.py
    └── py.typed


Another setting that you can enable is `last_expr_or_assign`. This makes it such that even if the last line of a cell is an assignment, the value of the assignment is displayed. 

In [7]:

from IPython.core.interactiveshell import InteractiveShell

# `ast_node_interactivity` is a setting that determines how the return value of the last line in a cell is displayed
# with `last_expr_or_assign`, the return value of the last expression is displayed unless it is assigned to a variable
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

With this setting, you don't have to repeat the name of the variable at the end of every cell to see the value of the variable.

In [8]:
import pandas as pd

df = pd.read_csv("./data/title.basics.tsv.gz", sep="\t", nrows=500).head()
# df ## this line is not required to see the value of df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short


## Panel Components

`panel` is built on top of [`param`](https://param.holoviz.org/). One useful way to think about `panel` and `param` is that `param` is a way to define state and `panel` is a way to visualize that state. And making state driven components is a great way to make interactive dashboards.

With `param`, you can define the state using a class based approach:

In [9]:
import param as pm


class MoviesStateExample(pm.Parameterized):
    name = pm.String()
    year = pm.Integer()

m = MoviesStateExample(name = "Goodfellas", year = 1990)

MoviesStateExample(name='Goodfellas', year=1990)

And with `panel`, you can visualize the state:

In [10]:
import panel as pn

pn.param.Param(m)

Param(MoviesStateExample, name='Goodfellas', sizing_mode='stretch_width')

When making any UI it is important to identify the state variables of a component. This usually involves understanding a few different things:

1. What are the inputs to the component?
2. What are the derived properties of the component?
3. What are the outputs of the component?



This typically forms a unidirectional graph where the inputs are used to derive the properties and the properties are used to derive the outputs.

### Identifying Inputs

Let's say we want to filter based on the year, the average ratings and the runtime of movies. In this case, we would have one variable for the input dataframe; and variables for the year, ratings and runtime ranges.

In [11]:
class MoviesStateExample(pm.Parameterized):
    input_df = pm.DataFrame()
    
    year_range = pm.Range()
    ratings_range = pm.Range()
    runtime_range = pm.Range()

### Identifying Derived Properties

In this example, when the class is initialized, we may want to load the CSV files, preprocess and clean the data.
When `self.df = df` is called, `param` will trigger an action with the name of the parameter. And any functions that are listening to that action will be called. We can use this feature to update any derived properties.

There are a few different ways to listen to changes in a parameter.

1. Add a member function with the `param.depends` decorator.

In [12]:
class MovieUsingDepends(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.df = pd.read_csv("./data/title.basics.tsv.gz", sep="\t", nrows=500)

    @pm.depends("df", watch=True)
    def _update_bounds(self):
        self.start_year = int(self.df["startYear"].min())
        self.end_year = int(self.df["startYear"].max())


m = MovieUsingDepends()
print(dict(start=m.start_year, end=m.end_year))

{'start': 1892, 'end': 1912}


2. Use `pm.bind` with the `watch=True` argument.

In [13]:
class MovieUsingBind(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        pm.bind(self._update_bounds, self.param.df, watch=True)
        self.df = pd.read_csv("./data/title.basics.tsv.gz", sep="\t", nrows=500)

    def _update_bounds(self, df):
        self.start_year = int(self.df["startYear"].min())
        self.end_year = int(self.df["startYear"].max())


m = MovieUsingBind()
print(dict(start=m.start_year, end=m.end_year))

{'start': 1892, 'end': 1912}


3. Use `self.param.watch` in the `__init__` function.

In [14]:
class MovieUsingWatch(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.param.watch(self._update_bounds, "df")
        self.df = pd.read_csv("./data/title.basics.tsv.gz", sep="\t", nrows=500)

    def _update_bounds(self, event):
        if event.name == "df":
            df = event.new
            self.start_year = int(self.df["startYear"].min())
            self.end_year = int(self.df["startYear"].max())


m = MovieUsingWatch()
print(dict(start=m.start_year, end=m.end_year))

{'start': 1892, 'end': 1912}



::: {.callout-tip}

`@param.depends` decorator with `watch=True` is the simplest because it is more explicit and easier to read.

:::

In all cases, when you use `watch=True`, you have created a dependent function, and any properties that are updated in that function are dependent properties. 

### Outputs

Finally, it is important to define the outputs of the component. In this case, the outputs is a derived component that represents the filtered dataframe. 

In [15]:
class MovieWithOutputs(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.df = pd.read_csv(
            "./data/title.basics.tsv.gz",
            sep="\t",
            usecols=["primaryTitle", "startYear"],
            nrows=200,
        )

    @pm.depends("df", watch=True)
    def _update_bounds(self):
        self.start_year = int(self.df["startYear"].min())
        self.end_year = int(self.df["startYear"].max())

    @pm.depends("df", "start_year", "end_year")
    def output_df(self):
        df, start_year, end_year = (
            self.df,
            self.start_year,
            self.end_year,
        )
        return df.query(f"startYear >= {start_year}").query(f"startYear <= {end_year}")


m = MovieWithOutputs()
m.output_df()

,primaryTitle,startYear
0,Carmencita,1894
1,Le clown et ses chiens,1892
2,Poor Pierrot,1892
3,Un bon bock,1892
4,Blacksmith Scene,1893
...,...,...
195,La fuite en Égypte,1898
196,Glasgow Fire Engine,1898
197,Gran corrida de toros,1898
198,Indian War Council,1894


Storing the outputs as a function allows for easy access to the outputs for testing, debugging and for use in other components.

On some occassions however, you might want to store the outputs as a property to cache outputs. This is useful when the output is expensive to compute.

In [16]:
class MovieWithDependentOutputs(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()
    filtered_df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.df = pd.read_csv(
            "./data/title.basics.tsv.gz",
            sep="\t",
            usecols=["primaryTitle", "startYear"],
            nrows=200,
        )

    @pm.depends("df", watch=True)
    def _update_bounds(self):
        self.start_year = int(self.df["startYear"].min())
        self.end_year = int(self.df["startYear"].max())

    @pm.depends("df", "start_year", "end_year", watch=True)
    def _output_df(self):
        df, start_year, end_year = (
            self.df,
            self.start_year,
            self.end_year,
        )
        self.filtered_df = df.query(f"startYear >= {start_year}").query(
            f"startYear <= {end_year}"
        )
        return self.filtered_df


m = MovieWithDependentOutputs()
m.filtered_df

,primaryTitle,startYear
0,Carmencita,1894
1,Le clown et ses chiens,1892
2,Poor Pierrot,1892
3,Un bon bock,1892
4,Blacksmith Scene,1893
...,...,...
195,La fuite en Égypte,1898
196,Glasgow Fire Engine,1898
197,Gran corrida de toros,1898
198,Indian War Council,1894


## View

After setting up the state of an component, we can create a view that presents the state, derived values and the outputs. 

One convention is to make this part of a `panel()` or a `view()` method that returns the layout and initializes any `pn.widgets` in and only in this method. 


In [17]:
class MoviesPanel(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()
    filtered_df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        basics = pd.read_csv("./data/title.basics.tsv.gz", sep="\t", nrows=500)
        ratings = pd.read_csv("./data/title.ratings.tsv.gz", sep="\t", nrows=500)
        self.df = basics.merge(ratings, on="tconst").dropna()

    @pm.depends("df", watch=True)
    def _update_bounds(self):
        self.start_year = int(self.df["startYear"].min())
        self.end_year = int(self.df["startYear"].max())

    @pm.depends("df", "start_year", "end_year", watch=True)
    def _output_df(self):
        df, start_year, end_year = (
            self.df,
            self.start_year,
            self.end_year,
        )
        self.filtered_df = df.query(f"startYear >= {start_year}").query(
            f"startYear <= {end_year}"
        )
        return self.filtered_df

    def panel(self):
        return pn.Column(
            pn.Row(
                pn.widgets.IntInput.from_param(self.param.start_year, name="Start Year"),
                pn.widgets.IntInput.from_param(self.param.end_year, name="End Year"),
            ),
            pn.widgets.Tabulator.from_param(
                self.param.filtered_df, pagination="remote", page_size=5
            ),
        )


m = MoviesPanel()
m.panel()

Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] IntInput(name='Start Year', sizing_mode='stretch_width', value=1892)
        [1] IntInput(name='End Year', sizing_mode='stretch_width', value=1912)
    [1] Tabulator(name='Filtered df', page_size=5, pagination='remote', sizing_mode='stretch_width', value=        tconst titleType  ...)

When a user updates the start year or the end year, the filtered dataframe is updated. And when that filtered data is updated, the `Tabulator` widget is updated. This is a very simple example, but this pattern can be extended to more complex dashboards.

::: callout-note

It is possible to also derive from `pn.viewable.Viewer` and implement the `__panel__` method. Doing this will return the layout of the component when the component is displayed in a notebook.

:::

Every widget in panel accepts a few different types of arguments (See [this section in the official documentation](https://panel.holoviz.org/tutorials/intermediate/interactivity.html#declarative) for more information). But the two most common types are:

### A parameter instance

It is important to understand the difference between an attribute instance and a parameter instance. An attribute instance is the value that most people typically deal with.

In [18]:
m.start_year

1892

A parameter instance on the other hand is a special instance created by any class that inherits from `param.Parameterized`.

In [19]:
m.param.start_year

In [20]:
isinstance(m.param.start_year, pm.Parameter)

True

Attribute instances are implemented using descriptors. When the Python interpreter sees `m.start_year`, it will return the value of the attribute by called the `__get__` method of the `param.Integer` descriptor. This returns the current state at the time of the invocation.

In [21]:
type(m.start_year)

int

`m.param.start_year` on the other hand is a special value that contains a `pm.Parameter` instance that allows enabling the reactive features. This special value has to be passed into a widget to create a two way binding, and this has to be done using the `from_param` method.

In [22]:
pn.widgets.IntInput.from_param(m.param.start_year)

IntInput(name='Start year', sizing_mode='stretch_width', value=1892)

### A method using `pn.bind` or `pn.depends` that returns a value

1. Using `pn.bind`

Widgets also can accept a method that is bound to a parameter. This is useful when you want to create a widget that is dependent on another parameter.

In [23]:
def _get_dataframe(df):
    return df.head().drop(columns=["tconst", "titleType", "isAdult", "endYear", "genres", "originalTitle"])
    
pn.widgets.Tabulator(pn.bind(_get_dataframe, m.param.filtered_df))

Tabulator(sizing_mode='stretch_width', value=             p...)

`pn.bind` is a way to bind a method to a parameter, and this method will be called whenever the parameter changes. 

Keep in mind that `pn.bind` returns a function:

In [24]:
type(pn.bind(_get_dataframe, m.param.filtered_df))

function

And calling that function will return the value of the parameter at that time.

In [25]:
f = pn.bind(_get_dataframe, m.param.filtered_df);

f()

,primaryTitle,startYear,runtimeMinutes,averageRating,numVotes
0,Carmencita,1894,1,5.7,2115
1,Le clown et ses chiens,1892,5,5.6,285
2,Poor Pierrot,1892,5,6.4,2148
3,Un bon bock,1892,12,5.3,183
4,Blacksmith Scene,1893,1,6.2,2872


But in Jupyter notebooks, even displaying the function will call the function and return the value.

In [26]:
f

<function param.reactive.bind.<locals>.wrapped(*wargs, **wkwargs)>

2. Using `@pm.depends` or `@pn.depends`

The input to a widget can also be a function or a method that is part of the class that uses the `@pm.depends` decorator.
If this decorator is used, and when this method is passed to a widget, the method will be called whenever the parameter changes and the return value is used to update the UI.

In this case however, displaying the method will not call the method:

In [27]:
@pm.depends(m.param.filtered_df)
def _get_dataframe(filtered_df):
    return df.head().drop(columns=["tconst", "titleType", "isAdult", "endYear", "genres", "originalTitle"])

_get_dataframe

<function __main__._get_dataframe(filtered_df)>

In [28]:
_get_dataframe(m.filtered_df)

,primaryTitle,startYear,runtimeMinutes
0,Carmencita,1894,1
1,Le clown et ses chiens,1892,5
2,Poor Pierrot,1892,5
3,Un bon bock,1892,12
4,Blacksmith Scene,1893,1


In [29]:
class MoviesPanel(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()
    filtered_df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        basics = pd.read_csv("./data/title.basics.tsv.gz", sep="\t", nrows=500)
        ratings = pd.read_csv("./data/title.ratings.tsv.gz", sep="\t", nrows=500)
        self.df = basics.merge(ratings, on="tconst").dropna()

    @pm.depends("df", watch=True)
    def _update_bounds(self):
        self.start_year = int(self.df["startYear"].min())
        self.end_year = int(self.df["startYear"].max())

    @pm.depends("df", "start_year", "end_year", watch=True)
    def _output_df(self):
        df, start_year, end_year = (
            self.df,
            self.start_year,
            self.end_year,
        )
        self.filtered_df = df.query(f"startYear >= {start_year}").query(
            f"startYear <= {end_year}"
        )
        return self.filtered_df

    @pm.depends("filtered_df")
    def _get_dataframe(filtered_df):
        return df.head().drop(columns=["tconst", "titleType", "isAdult", "endYear", "genres", "originalTitle"])

    def panel(self):
        return pn.Column(
            pn.Row(
                pn.widgets.IntInput.from_param(self.param.start_year, name="Start Year"),
                pn.widgets.IntInput.from_param(self.param.end_year, name="End Year"),
            ),
            pn.widgets.Tabulator(
                self._get_dataframe, pagination="remote", page_size=5
            ),
        )



m = MoviesPanel()
m.panel()

Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] IntInput(name='Start Year', sizing_mode='stretch_width', value=1892)
        [1] IntInput(name='End Year', sizing_mode='stretch_width', value=1912)
    [1] Tabulator(page_size=5, pagination='remote', sizing_mode='stretch_width', value=             p...)

::: {.callout-note}

Notice that the `@pm.depends` decorator is used to define the method `_get_dataframe` does not use the `watch=True` argument. 

If the `@pm.depends` decorator uses the `watch=True` argument, the method will be called whenever the parameter changes AND the method will be called again when the widget needs to be updated.

:::


::: {.callout-tip}

When `watch=True` is used, it is recommended to not return anything from that method and simply update the state of the class instead. It's easier to reason about the code and forces using `from_param` to bind to a derived property instead.

:::

Using these two ways, you can make any reactive component using panel by using two way bindings to update the state and the UI, and trigger updates to other dependencies which in turn can update other parts of the UI.

## Example: Movies Dashboard

In [30]:
#| code-fold: true

import movies_dashboard

We can instantiate the `Movies` class.

In [31]:
m = movies_dashboard.movies.Movies();

And call the `panel()` method to get an interactive dashboard.

In [32]:
m.panel()

Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] RangeSlider(end=10, name='Ratings range', sizing_mode='stretch_width', step=1, value=(0, 10), value_end=10)
        [1] RangeSlider(end=2025, name='Year range', sizing_mode='stretch_width', start=1894, step=1, value=(1894, 2025), value_end=2025, value_start=1894)
        [2] RangeSlider(end=59460, name='Runtime range', sizing_mode='stretch_width', start=1, step=1, value=(1, 59460), value_end=59460, value_start=1)
    [1] Row(sizing_mode='stretch_width')
        [0] Select(name='Genre', options=OrderedDict({'Romance': 'R...]), sizing_mode='stretch_width', value='Romance')
    [2] Tabs(sizing_mode='stretch_width')
        [0] HoloViews(DynamicMap, height=400, name='Visualization', sizing_mode='fixed', width=900)
        [1] Tabulator(page_size=10, pagination='remote', sizing_mode='stretch_width', value=           tconst t...)

By having methods that return subcomponents, we can easily combine them on the fly on a case by case basis.

In [33]:
#| code-fold: true
m.genre = "Documentary"
documentary_plot = m._update_plot()
m.genre = "Comedy"
comedy_plot = m._update_plot()

m.genre = "Romance"

(documentary_plot + comedy_plot).cols(1)

:Layout
   .BoxWhisker.I  :BoxWhisker   [startYear]   (averageRating)
   .BoxWhisker.II :BoxWhisker   [startYear]   (averageRating)

The only real disadvantage of this package based approach is that it is not straightforward to make a wasm only version of the panel dashboard, since that requires it to be all in one file

This approach allows making a package, using `param` to define dependent state, create UI when required in a class based approach. In my opinion, this lends itself to better testing, documentation and maintainability in the long run. This is also the approach recommended in the intermediate sections of the `panel` tutorials. Check out these links for more information:

- https://panel.holoviz.org/tutorials/intermediate/parameters.html
- https://panel.holoviz.org/tutorials/intermediate/interactivity.html
- https://panel.holoviz.org/tutorials/intermediate/reusable_components.html